In [32]:
# TFlearn helper to setup neural networks
import tflearn
from tflearn.data_utils import to_categorical, pad_sequences

# Tools to scrape the web
import re
import urllib
from bs4 import BeautifulSoup

# Data wrangling modules
import numpy as np
import pandas as pd

In [2]:
# Read ign data

ignd = pd.read_csv('ign.csv', index_col=0)

In [54]:
def get_review_verdict(game_url):
    url = 'http://ca.ign.com%s' % (game_url)
    
    # Get to the game page and extract link to review
    page = urllib.request.urlopen(url)
    soup = BeautifulSoup(page.read(), 'lxml')
    
    # Get the review link
    rev_tag = soup.find_all(name="a", class_="reviewLink", string=re.compile('Review'))[0]
    try:
        rev_url = rev_tag.attrs['href']
    
        # Extract the verdict of the review
        page = urllib.request.urlopen(rev_url)
        soup = BeautifulSoup(page.read(), 'lxml')
        verdict_tag = soup.find(name='div', class_='articleSubHeader', string=re.compile('The Verdict'))
        
        verdict = verdict_tag.next_sibling.next_sibling.next_sibling
        verdict_str = ''.join([s for s in verdict.strings])
    except:
        verdict_str = None
    
    return verdict_str

In [57]:
get_review_verdict(ignd.iloc[0].url)

"LittleBigPlanet PS Vita is the definitive LittleBigPlanet game. It's everything you loved (or possibly didn't) from the past games boiled down into a package you can play anywhere at any time. You can collect prize bubbles while watching TV, download user-created levels at home and then play them on a plane, and sink the hours into learning create mode via 10-minute chunks at the Laundromat. Then, there's all the new stuff like touch controls, games that don't involve Sackboy, and the creation potential that could give you an endless supply of free games. "

In [29]:
res = soup.find_all(name="a", class_="reviewLink", string=re.compile('Review'))[0]

In [33]:
url = res.attrs['href']
page = urllib.request.urlopen(url)
soup = BeautifulSoup(page.read(), 'lxml')

In [34]:
soup

<!DOCTYPE html>
<html class="locale-ca" lang="en">
<head prefix="og: http://ogp.me/ns# fb: http://www.facebook.com/2008/fbml">
<meta content="width=device-width" name="viewport"/>
<link href="http://platform.twitter.com" rel="dns-prefetch"/>
<!-- Optimizely -->
<meta data-project-id="1369090036" id="ign-optimizely"/>
<title>Way of the Samurai 4 Review - IGN</title>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="Way of the Samurai 4 places gamers in an open world begging to be explored. Too bad it's just not that fun." name="description"/>
<meta content="noodp, noydir" name="robots"/>
<meta content="IGN Entertainment, Inc." name="copyright"/>
<link href="http://www.ign.com/articles/2012/09/03/way-of-the-samurai-4-review" rel="canonical"/>
<link href="http://m.ca.ign.com/articles/2012/09/03/way-of-the-samurai-4-review" media="only screen and (max-width: 640px)" rel="alternate"/>
<link href="http://au.ign.com/articles/2012/09/03/way-of-the-samurai-4-re

In [39]:
verdict_tag = soup.find(name='div', class_='articleSubHeader', string=re.compile('The Verdict'))

In [46]:
verdict = verdict_tag.next_sibling.next_sibling.next_sibling

In [53]:
''.join([s for s in verdict.strings])

"With the freedom of an open world combined with inconsistent tone, it’s very hard to tell just what the development team at ACQUIRE Corp was aiming for.  While the story has branching paths, none were especially engrossing or entertaining, and the repetitive combat quickly took its toll. After fighting for honor, for myself and just for the sake of causing chaos in a tiny Japanese village, Way of the Samurai 4 did show me that I would not have been a very well liked person in Japan in the 1850s. But if the real world back then was anything like this game world, I wouldn't have wanted to spend much time there anyway."